In [1]:
### HAND DIGIT RECOGNITION ###
import cv2 as cv
import mediapipe as mp

In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)
mpDraw = mp.solutions.drawing_utils

In [3]:
TIPS = [4, 8, 12, 16, 20]

def find_digit(p):
    d = 0
    TIPS = [4, 8, 12, 16, 20]
    # THUMB POSITION DETECTION
    if p[12].x > p[20].x:
        if p[TIPS[0]].x > p[TIPS[0]-1].x:
            d += 1
    else:
        if p[TIPS[0]].x < p[TIPS[0]-1].x:
            d += 1
    for tip in TIPS[1:]:
        if p[tip].y < p[tip-2].y:
            d += 1
    if p[12].x > p[20].x:
        if (sum([p[6].x, p[10].x, p[14].x, p[18].x]) / 4) > p[4].x:
            d = 'ThumbsUp'
    else:
        if (sum([p[6].x, p[10].x, p[14].x, p[18].x]) / 4) < p[4].x:
            d = 'ThumbsUp'
    return d

In [4]:
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 1080)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720)
while True:
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        if results.multi_hand_landmarks:
            single_hand = results.multi_hand_landmarks[0]
            keypoints = single_hand.landmark
            digit = find_digit(keypoints)
            mpDraw.draw_landmarks(
                frame, 
                single_hand,
                mpHands.HAND_CONNECTIONS
            )
            cv.putText(frame, 'Digit - {0}'.format(digit),
                       (10, 70), cv.FONT_HERSHEY_COMPLEX, 2,
                       [0, 255, 0], 4)
        cv.imshow('HandDetection', frame)
    else:
        break
    k = cv.waitKey(1)
    if k == 113:
        break
cv.destroyAllWindows()